In [1]:
from qiskit_machine_learning.datasets import ad_hoc_data

adhoc_dimension = 2
train_features, train_labels, test_features, test_labels, adhoc_total = ad_hoc_data(
    training_size=60,
    test_size=15,
    n=adhoc_dimension,
    gap=0.3,
    plot_data=False,
    one_hot=False,
    include_sample_total=True,
)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [4]:
def reducir_muestras(dataset, porcentaje_mantener):
    
    if not 0 < porcentaje_mantener <= 1:
        raise ValueError('El porcentaje a mantener debe ser un número entre 0 y 1')
    
    #Calculamos el número de muestras a mantener
    num_muestras = int(len(dataset) * porcentaje_mantener)
    
    #Seleccionamos aleatoriamente las muestras a mantener
    indices = np.random.choice(dataset.index, num_muestras, replace=False)
    
    return dataset.loc[indices]
    
porcentaje_mantener = 1

data = pd.read_csv('diabetes.csv')
data_R = reducir_muestras(data, porcentaje_mantener)

In [5]:
X = data_R.iloc[:, :-1].values
y = data_R['Outcome'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

#pca = PCA(n_components=4)
#X = pca.fit_transform(X)

train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=2)

In [6]:
from qiskit_ibm_runtime import Session, SamplerV2, EstimatorV2
from qiskit.providers.fake_provider import Fake127QPulseV1
from qiskit_ibm_runtime.fake_provider import FakeBrisbane 
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator, QasmSimulator
#from qiskit_aer.primitives import SamplerV2 as Sampler

from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap

num_qubits = train_features.shape[1]

backend = AerSimulator(method='statevector', device='CPU')
session = Session(backend=backend)
pass_manager = generate_preset_pass_manager(optimization_level=1, target=backend.target)

sampler = SamplerV2(mode=session)
fidelity = ComputeUncompute(sampler=sampler, pass_manager=pass_manager)

feature_map = ZZFeatureMap(feature_dimension=num_qubits)
qk = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

In [ ]:
from sklearn.svm import SVC
import time

start = time.time()
matrix_train = qk.evaluate(x_vec=train_features)

SVM = SVC(kernel="precomputed")

SVM.fit(matrix_train, train_labels)
stop = time.time()

train_time = stop - start


start = time.time()
matrix_test = qk.evaluate(x_vec=test_features, y_vec=train_features)
SVM_score = SVM.score(matrix_test, test_labels)
stop = time.time()

test_time = stop - start


print(f"Training Time: {train_time}")
print(f"Testing Time: {test_time}")
print(f"Total Time: {train_time + test_time}")
print(f"Precomputed kernel classification test score: {SVM_score}")

AlgorithmError: 'Sampler job failed!'